In [4]:
import pandas as pd
import numpy as np
from clickhouse_driver import Client

In [8]:
machid = 1

In [44]:
# machid=st.experimental_get_query_params()['machid'][0]
client = Client(host='localhost', settings={'use_numpy': True})
#data=client.query_dataframe("""SELECT * FROM  default.mach M left join default.cpu C on M.cpuid=
#C.cpuid where M.machid={}""".format(machid))
cols=['machid', 'Vendor', 'System', 'Cores', 'Chips', 'Memory',
       'MemoSize(GB)', 'MemoNum', 'Storage', 'DiskSize(TB)', 'SSD',
       'OperatingSystem', 'OS', 'OSVersion', 'KernelVersion', 'FileSystem',
       'Compiler', 'FC', 'FV', 'CC', 'CV', 'C++C', 'C++V', 'cpuid', 'cpu',
       'Processor', 'ProcessorMHz', 'CPU(s)Orderable', 'Parallel',
       'BasePointerSize', 'PeakPointerSize', 'CoresPerCPU', 'L1Dcache(KB)',
       'L1Icache(KB)', 'L2cache(MB)', 'L3cache(MB)', 'OtherCache']
#data.drop("C_cpuid",axis=1,inplace=True)
#data.columns=cols

In [45]:
se_col=['System', 'Cores', 'Chips', 'Memory',
       'MemoSize_GB_', 'MemoNum', 'Storage', 'DiskSize_TB_',
       'OperatingSystem',  'KernelVersion', 'FileSystem','Compiler', 'cpu']

In [46]:
#同CPU其他机子
data = client.query_dataframe("""SELECT * FROM  default.mach M1 ,(SELECT cpu FROM  default.mach t  where t.machid={}) 
                                 M2 where M1.cpu=M2.cpu""".format(machid))
idx = data[['System']].drop_duplicates()[:10].index
# data.loc[idx,se_col].style.highlight_max(subset=['Cores', 'Chips', 'MemoSize_GB_',  'DiskSize_TB_'])

In [49]:
#同厂商其他机子
data=client.query_dataframe("""SELECT * FROM  default.mach M1 ,(SELECT Vendor FROM  default.mach t  where t.machid={}) 
M2 where M1.Vendor=M2.Vendor""".format(machid))
idx=data[['System']].drop_duplicates()[:10].index
# data.loc[idx,se_col].style.highlight_max(subset=['Cores', 'Chips', 'MemoSize_GB_',  'DiskSize_TB_'])

In [51]:
# #同tag其他机子
# data=client.query_dataframe("""select * from mach M1, tags T ,(SELECT * FROM  mach  where mach.machid={}) 
# M2 where T.`L.C.Processor`=M2.cpu and M1.cpu=M2.cpu""".format(machid))
# idx=data[['System']].drop_duplicates().index

In [52]:
', '.join([i[0] for i in client.execute("SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE, COLUMN_DEFAULT FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name='tags'")])

'L.C.Processor, perl_tag, gnu_tag, route_tag, discrete_tag, xml_tag, video_tag, aichess_tag, aimc_tag, aisudoku_tag, compression_tag, explosion_tag, physics_tag, molecular_tag, biomedical_tag, ray_tag, fluid_tag, weather_tag, render_3D_tag, atmosphere_tag, climateocean_tag, image_tag, electro_tag, regionalocean_tag'

In [53]:
', '.join([i[0] for i in client.execute("SELECT COLUMN_NAME, DATA_TYPE, IS_NULLABLE, COLUMN_DEFAULT FROM INFORMATION_SCHEMA.COLUMNS WHERE table_name='mach'")])

'machid, Vendor, System, Cores, Chips, Memory, MemoSize(GB), MemoNum, Storage, DiskSize(TB), SSD, OperatingSystem, OS, OSVersion, KernelVersion, FileSystem, Compiler, FC, FV, CC, CV, C++C, C++V, cpuid, cpu'

In [68]:
tags_tmp = '''L.C.Processor, perl_tag, gnu_tag, route_tag, discrete_tag, xml_tag, video_tag, aichess_tag, aimc_tag, aisudoku_tag, compression_tag, explosion_tag, physics_tag, molecular_tag, biomedical_tag, ray_tag, fluid_tag, weather_tag, render_3D_tag, atmosphere_tag, climateocean_tag, image_tag, electro_tag, regionalocean_tag'''
tags_tmp = ', '.join(['T.`' + i + '`' for i in tags_tmp.split(', ')])

In [76]:
target_machid = 2
q = '''(select ''' + tags_tmp + ''' from mach M1 left outer join tags T on M1.cpu == T.`L.C.Processor` where M1.machid == {})'''.format(target_machid)
q

'(select T.`L.C.Processor`, T.`perl_tag`, T.`gnu_tag`, T.`route_tag`, T.`discrete_tag`, T.`xml_tag`, T.`video_tag`, T.`aichess_tag`, T.`aimc_tag`, T.`aisudoku_tag`, T.`compression_tag`, T.`explosion_tag`, T.`physics_tag`, T.`molecular_tag`, T.`biomedical_tag`, T.`ray_tag`, T.`fluid_tag`, T.`weather_tag`, T.`render_3D_tag`, T.`atmosphere_tag`, T.`climateocean_tag`, T.`image_tag`, T.`electro_tag`, T.`regionalocean_tag` from mach M1 left outer join tags T on M1.cpu == T.`L.C.Processor` where M1.machid == 2)'

In [77]:
tmp_l = client.execute(q)

In [78]:
len(tmp_l)

1

In [79]:
tmp_l

[array(['Intel Xeon Bronze 3206R', '0', '0', '0', '0', '0', '0', '0', '0',
        '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
        '0', '0'], dtype='<U23')]